In [ ]:
# Configure AWS profile for local development
%env AWS_PROFILE=platform-developer

In [ ]:
import pandas as pd

from adapters.axiell.steps.loader import AXIELL_NAMESPACE
from adapters.axiell.helpers import build_window_store

window_store = build_window_store(use_rest_api_table=False)

print("\nWindow status table info:")
print(window_store.table)

DISPLAY_LIMIT = 20

In [ ]:
def window_status_frame(limit: int = 25, state: str | None = None) -> pd.DataFrame:

    """Return recent window status rows from the IcebergWindowStore."""
    rows = list(window_store.load_status_map().values())

    if not rows:
        return pd.DataFrame(
            columns=[
                "window_key",
                "window_start",
                "window_end",
                "state",
                "attempts",
                "last_error",
                "record_ids",
                "updated_at",
            ]
        )

    df = pd.DataFrame(rows)

    for column in ("window_start", "window_end", "updated_at"):
        if column in df.columns:
            df[column] = pd.to_datetime(df[column])

    df = df.sort_values("window_start", ascending=False)

    if state is not None:
        df = df[df["state"] == state]

    if limit is not None:
        df = df.head(limit)

    return df.reset_index(drop=True)

window_status_frame(limit=10)

In [ ]:
from datetime import datetime, timedelta, timezone

# Example: Get windows from the last 24 hours
now = datetime.now(timezone.utc)
start_time = now - timedelta(hours=6)

print(f"Fetching windows from {start_time} to {now}...")
windows_in_range = window_store.list_in_range(start=start_time, end=now)

if windows_in_range:
    df_range = pd.DataFrame(windows_in_range)
    for col in ["window_start", "window_end", "updated_at"]:
        if col in df_range.columns:
            df_range[col] = pd.to_datetime(df_range[col])
    
    # Sort by window start
    df_range = df_range.sort_values("window_start", ascending=False)
    display(df_range)
else:
    print("No windows found in this range.")

In [ ]:
print("Fetching failed windows...")
failed_windows = window_store.list_by_state("failed")

if failed_windows:
    df_failed = pd.DataFrame(failed_windows)
    for col in ["window_start", "window_end", "updated_at"]:
        if col in df_failed.columns:
            df_failed[col] = pd.to_datetime(df_failed[col])
            
    df_failed = df_failed.sort_values("window_start", ascending=False)
    display(df_failed)
else:
    print("No failed windows found.")